In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Set random seed for reproducibility
np.random.seed(42)

# Define parameters for dataset generation
n_products = 10000
n_days = 365  # One year of daily data

# Generate product IDs
product_ids = [f"PROD-{i}" for i in range(1, n_products + 1)]

# Generate date range for the dataset
date_range = [datetime(2024, 1, 1) + timedelta(days=i) for i in range(n_days)]

# Generate synthetic inventory dataset
data = []
for product in product_ids:
    base_demand = np.random.randint(10, 200)  # Base daily demand per product
    trend = np.linspace(1, 1.5, n_days)  # Simulating a growth trend over time
    seasonality = 1 + 0.2 * np.sin(np.linspace(0, 2 * np.pi, n_days))  # Seasonality effect
    
    for day, date in enumerate(date_range):
        demand = int(base_demand * trend[day] * seasonality[day] * np.random.uniform(0.8, 1.2))
        stock_level = np.random.randint(50, 500)  # Random stock levels
        replenishment_lead_time = np.random.randint(1, 10)  # Lead time for stock arrival
        cost_per_unit = np.random.uniform(5, 50)  # Random product cost
        
        data.append([product, date, demand, stock_level, replenishment_lead_time, cost_per_unit])

# Create DataFrame
inventory_df = pd.DataFrame(data, columns=[
    "ProductID", "Date", "Demand", "StockLevel", "ReplenishmentLeadTime", "CostPerUnit"
])

inventory_df.head()

,ProductID,Date,Demand,StockLevel,ReplenishmentLeadTime,CostPerUnit
0,PROD-1,2024-01-01,125,320,8,31.939632
1,PROD-1,2024-01-02,97,264,8,20.016888
2,PROD-1,2024-01-03,96,180,6,7.538521
3,PROD-1,2024-01-04,123,463,6,5.035044
4,PROD-1,2024-01-05,136,210,6,5.317984


In [2]:
# Calculate KPIs

# 1. Stockout Rate
stockout_rate = inventory_df.groupby("ProductID").apply(
    lambda x: (x["StockLevel"] == 0).sum() / len(x) * 100
).reset_index(name="StockoutRate")


In [3]:
# 2. Demand Variability
demand_variability = inventory_df.groupby("ProductID")["Demand"].std().reset_index(name="DemandVariability")
demand_variability.head()

,ProductID,DemandVariability
0,PROD-1,19.705791
1,PROD-10,5.513071
2,PROD-100,32.755881
3,PROD-1000,12.964576
4,PROD-10000,18.052592


In [4]:
# 3. Reorder Point
reorder_point = inventory_df.groupby("ProductID").apply(
    lambda x: x["Demand"].mean() * x["ReplenishmentLeadTime"].mean() + 1.5 * x["Demand"].std()
).reset_index(name="ReorderPoint")
reorder_point.head()

,ProductID,ReorderPoint
0,PROD-1,718.045322
1,PROD-10,197.931644
2,PROD-100,1172.226184
3,PROD-1000,418.813349
4,PROD-10000,629.961756


In [5]:
# 4. Inventory Turnover Ratio
inventory_turnover = inventory_df.groupby("ProductID").apply(
    lambda x: x["Demand"].sum() / x["StockLevel"].mean()
).reset_index(name="InventoryTurnoverRatio")
inventory_turnover.head()

,ProductID,InventoryTurnoverRatio
0,PROD-1,185.010040
1,PROD-10,49.252554
2,PROD-100,304.146502
3,PROD-1000,111.208331
4,PROD-10000,155.409451


In [6]:
# 5. Fill Rate
fill_rate = inventory_df.groupby("ProductID").apply(
    lambda x: 1 - (x["StockLevel"] == 0).sum() / x["Demand"].count()
).reset_index(name="FillRate")
fill_rate.head()

,ProductID,FillRate
0,PROD-1,1.0
1,PROD-10,1.0
2,PROD-100,1.0
3,PROD-1000,1.0
4,PROD-10000,1.0


In [7]:
# 6. Economic Order Quantity (EOQ)
ordering_cost = 50  # Assumed fixed ordering cost per order
holding_cost_per_unit = 0.2  # Assumed holding cost per unit
eoq = inventory_df.groupby("ProductID").apply(
    lambda x: np.sqrt((2 * x["Demand"].sum() * ordering_cost) / (holding_cost_per_unit * x["CostPerUnit"].mean()))
).reset_index(name="EOQ")
print(ordering_cost)

50


In [8]:
# 7. Days of Supply
days_of_supply = inventory_df.groupby("ProductID").apply(
    lambda x: x["StockLevel"].mean() / x["Demand"].mean()
).reset_index(name="DaysOfSupply")
days_of_supply.head()

,ProductID,DaysOfSupply
0,PROD-1,1.972866
1,PROD-10,7.410783
2,PROD-100,1.200080
3,PROD-1000,3.282128
4,PROD-10000,2.348635


In [9]:
# 8. Dead Stock Percentage
dead_stock = inventory_df.groupby("ProductID").apply(
    lambda x: (x["StockLevel"] > x["Demand"].max()).sum() / len(x) * 100
).reset_index(name="DeadStockPercentage")
dead_stock.head()

,ProductID,DeadStockPercentage
0,PROD-1,66.301370
1,PROD-10,99.726027
2,PROD-100,41.095890
3,PROD-1000,84.657534
4,PROD-10000,79.178082


In [10]:
# 9. Revenue Lost Due to Stockouts
revenue_lost = inventory_df.groupby("ProductID").apply(
    lambda x: (x["StockLevel"] == 0).sum() * x["CostPerUnit"].mean()
).reset_index(name="RevenueLostStockouts")
revenue_lost.head()

,ProductID,RevenueLostStockouts
0,PROD-1,0.0
1,PROD-10,0.0
2,PROD-100,0.0
3,PROD-1000,0.0
4,PROD-10000,0.0


In [11]:
# 10. Safety Stock Calculation
safety_stock = inventory_df.groupby("ProductID").apply(
    lambda x: 1.65 * x["Demand"].std() * np.sqrt(x["ReplenishmentLeadTime"].mean())
).reset_index(name="SafetyStock")
safety_stock.head()

,ProductID,SafetyStock
0,PROD-1,72.724672
1,PROD-10,20.429511
2,PROD-100,122.006584
3,PROD-1000,45.893426
4,PROD-10000,66.055550


In [12]:
# Merge all KPIs into a single DataFrame
kpi_df = stockout_rate.merge(demand_variability, on="ProductID") \
    .merge(reorder_point, on="ProductID") \
    .merge(inventory_turnover, on="ProductID") \
    .merge(fill_rate, on="ProductID") \
    .merge(eoq, on="ProductID") \
    .merge(days_of_supply, on="ProductID") \
    .merge(dead_stock, on="ProductID") \
    .merge(revenue_lost, on="ProductID") \
    .merge(safety_stock, on="ProductID")

# Save dataset and KPIs to CSV
inventory_df.to_csv("inventory_dataset.csv", index=False)
kpi_df.to_csv("inventory_kpi_results.csv", index=False)

# Display final DataFrame
print("Synthetic Inventory Dataset and KPI results saved successfully.")

Synthetic Inventory Dataset and KPI results saved successfully.
